In [400]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from scipy.interpolate import RegularGridInterpolator

from pathlib import Path
from MITRotor import BEM, IEA10MW, IEA15MW, BEMGeometry, AerodynamicProperties, NoTipLoss, PrandtlTipLoss, ConstantInduction, KraghAerodynamics, ClassicalMomentum, UnifiedMomentum, MadsenMomentum, NoTangentialInduction, DefaultTangentialInduction, BEMSolution


In [401]:
def per_error(A, E):
    error = abs((A - E) / E) * 100

    return error

In [402]:
casenames = [r's0_v4']

wrfles_data = []
for count, name in enumerate(casenames):
    wrfles_data.append(dict(np.load('/scratch/09909/smata/wrf_les_sweep/runs/old_clockwise/gad_sweep/'+casenames[count]+'.npz')))

count = 0


In [403]:
y = (wrfles_data[count]['Y3'] - wrfles_data[count]['rotor_yloc'])/(wrfles_data[count]['diameter']/2)
z = (wrfles_data[count]['Z3'] - wrfles_data[count]['hub_height'])/(wrfles_data[count]['diameter']/2)

In [404]:
r     = (y**2 + z**2)**(1/2)
theta = np.arctan2(y,z)

In [405]:
wsn4  = ((np.mean(wrfles_data[count]['ux_n4D'], axis=0)**2 + np.mean(wrfles_data[count]['vx_n4D'], axis=0)**2)**(1/2) / wrfles_data[count]['uinf'])
wdn4  = (np.arctan2(np.mean(wrfles_data[count]['vx_n4D'], axis=0), np.mean(wrfles_data[count]['ux_n4D'], axis=0)))

ws  = ((np.mean(wrfles_data[count]['ux_0D'], axis=0)**2 + np.mean(wrfles_data[count]['vx_0D'], axis=0)**2)**(1/2) / wrfles_data[count]['uinf'])
wd  = (np.arctan2(np.mean(wrfles_data[count]['vx_0D'], axis=0), np.mean(wrfles_data[count]['ux_0D'], axis=0)))

ct  = np.mean(wrfles_data[count]['ct'],  axis=0)
cp  = np.mean(wrfles_data[count]['cp'],  axis=0)
cl  = np.mean(wrfles_data[count]['cl'],  axis=0)
cd  = np.mean(wrfles_data[count]['cd'],  axis=0)
aoa = np.mean(wrfles_data[count]['aoa'], axis=0)
f   = np.mean(wrfles_data[count]['f'],   axis=0)
fn  = np.mean(wrfles_data[count]['fn'],  axis=0)
ft  = np.mean(wrfles_data[count]['ft'],  axis=0)
vd  = np.mean(wrfles_data[count]['v1'],  axis=0)

In [406]:
Ad = np.pi * (wrfles_data[count]['diameter']/2)**2

In [407]:
# Mask points where r > R
mask = r <= 1.5
r_filtered     = r[mask]
theta_filtered = theta[mask]
ws_filtered    = ws[mask]
wd_filtered    = wd[mask]

In [408]:
def interp_polars(r,t,R,T,data):
    interpolator = RegularGridInterpolator(
        (r, t), 
        data, 
        bounds_error=False, 
        fill_value=None  # Enables extrapolation
    )

    # Interpolation points
    points_new = np.column_stack((R.ravel(), T.ravel()))

    # Interpolate to new points
    return interpolator(points_new).reshape(R.shape)

In [409]:
# Interpolate to new polar grid
r_new = np.linspace(0.0, 0.999, 60)  # 50 points in r
theta_new = np.linspace(0, 2 * np.pi, 360)  # 100 points in theta
Theta_new, R_new = np.meshgrid(theta_new, r_new)

In [410]:
# Convert new polar grid to Cartesian for interpolation
X_new = R_new * np.sin(Theta_new)
Y_new = R_new * np.cos(Theta_new)

In [411]:
# Interpolate data to the new polar grid
ws_rt = griddata(
    points=(y[mask], z[mask]),
    values=ws_filtered,
    xi=(X_new, Y_new),
    method='linear'
)
# Interpolate data to the new polar grid
wd_rt = griddata(
    points=(y[mask], z[mask]),
    values=wd_filtered,
    xi=(X_new, Y_new),
    method='linear'
)

In [412]:
cl_r = wrfles_data[count]['rOverR']
# cl_t = np.linspace(0, 2 * np.pi, 158) + np.pi/2
cl_t = np.linspace(0, 2 * np.pi, 158)

cl_T, cl_R = np.meshgrid(cl_t, cl_r)

cl_x = cl_R * np.sin(cl_T)
cl_y = cl_R * np.cos(cl_T)

cl_r_new = np.linspace(0.0, 0.98, 60)
# cl_t_new = np.linspace(0, 2 * np.pi, 360) + np.pi/2
cl_t_new = np.linspace(0, 2 * np.pi, 360)

cl_T_new, cl_R_new = np.meshgrid(cl_t_new, cl_r_new)

cl_x_new = cl_R_new * np.cos(cl_T_new)
cl_y_new = cl_R_new * np.sin(cl_T_new)

In [413]:
# Mask points where r > R
mask = r <= 1.5
r_filtered     = r[mask]
theta_filtered = theta[mask]
ws_filtered    = ws[mask]
wd_filtered    = wd[mask]

In [414]:
def interp_polars(r,t,R,T,data):
    interpolator = RegularGridInterpolator(
        (r, t), 
        data, 
        bounds_error=False, 
        fill_value=None  # Enables extrapolation
    )

    # Interpolation points
    points_new = np.column_stack((R.ravel(), T.ravel()))

    # Interpolate to new points
    return interpolator(points_new).reshape(R.shape)

In [415]:
# Interpolate to new polar grid
r_new = np.linspace(0.0, 0.99999, 60)  # 50 points in r
theta_new = np.linspace(0, 2 * np.pi, 360)  # 100 points in theta
Theta_new, R_new = np.meshgrid(theta_new, r_new)

In [416]:
# Convert new polar grid to Cartesian for interpolation
X_new = R_new * np.sin(Theta_new)
Y_new = R_new * np.cos(Theta_new)

In [417]:
# Interpolate data to the new polar grid
ws_rt = griddata(
    points=(y[mask], z[mask]),
    values=ws_filtered,
    xi=(X_new, Y_new),
    method='linear'
)
# Interpolate data to the new polar grid
wd_rt = griddata(
    points=(y[mask], z[mask]),
    values=wd_filtered,
    xi=(X_new, Y_new),
    method='linear'
)

In [418]:
# Initialize rotor with increased radial resolution.
rotor = IEA10MW()

bem = BEM(rotor=rotor, geometry=BEMGeometry(Nr=60, Ntheta=360), aerodynamic_model=KraghAerodynamics(), tiploss_model=NoTipLoss(), momentum_model=ConstantInduction(), tangential_induction_model=NoTangentialInduction())

In [419]:
pitch, tsr, yaw = np.deg2rad(0), 10.634, np.deg2rad(0.0)
sol = bem(pitch, tsr, yaw, U=ws_rt, wdir=wd_rt)

In [420]:
# aoa_rt  = interp_polars(cl_r,cl_t,cl_R_new,cl_T_new,vd)
# aoa_rt  = interp_polars(cl_r,cl_t,cl_R_new,cl_T_new,ws_rt * np.cos(wd_rt))
aoa_rt  = ws_rt * wrfles_data[count]['uinf'] * np.cos(wd_rt)


In [421]:
wrf_ind = 1/Ad * np.trapezoid(np.trapezoid((1- aoa_rt.T/wrfles_data[count]['uinf']) * cl_R_new[:,0] * (wrfles_data[count]['diameter']/2), cl_R_new[:,0] * (wrfles_data[count]['diameter']/2)), cl_T_new[0,:])

In [422]:
mit_ind = 1/Ad * np.trapezoid(np.trapezoid((1- (sol.U('sector') * np.cos(sol.wdir('sector'))).T) * sol.geom.mu * (wrfles_data[count]['diameter']/2), sol.geom.mu * (wrfles_data[count]['diameter']/2)), sol.geom.theta)

In [423]:
#MIT
mit_ref = 0.3755722857166954
per_error(mit_ref, mit_ind)

np.float64(11.559681308232147)

In [424]:
#WRF
wrf_ref = 0.3607068373029895
per_error(wrf_ref, wrf_ind)

np.float64(11.559681308232134)

In [425]:
wrf_thr = wrfles_data[count]['thrust'].mean()
mit_thr = 1/Ad * np.trapezoid(np.trapezoid(((sol.Ct('sector')).T) * sol.geom.mu * (wrfles_data[count]['diameter']/2), sol.geom.mu * (wrfles_data[count]['diameter']/2)), sol.geom.theta)

In [426]:
#MIT
mit_ref = 0.914400610308625
per_error(mit_ref, mit_thr)

np.float64(3.0991338205585954)

In [427]:
#WRF
wrf_ref = 810548.4
per_error(wrf_ref, wrf_thr)

np.float32(3.4157043)

In [428]:
wrf_pow = wrfles_data[count]['power_aero'].mean()
mit_pow = 1/Ad * np.trapezoid(np.trapezoid(((sol.Cp('sector')).T) * sol.geom.mu * (wrfles_data[count]['diameter']/2), sol.geom.mu * (wrfles_data[count]['diameter']/2)), sol.geom.theta)

In [429]:
#MIT
mit_ref = 0.4901736756170172
per_error(mit_ref, mit_pow)

np.float64(15.919092318589428)

In [430]:
#WRF
wrf_ref = 3137528.8
per_error(wrf_ref, wrf_pow)

np.float32(16.144428)